# 1. Introduction
This notebook is used to develop the scanning code for the scanner minimimum viable product (MVP). Each section will demostrate a different feature of the scanner except the final section which will combine the previous sections to create a fully functional scanning code. The final section will then be copied over to a separate python module. The image processing will be done in a separate notebook and subsequent module.

In [2]:
import cv2
import numpy as np
import sys
import keyboard
import time
import os
import glob
import matplotlib.pyplot as plt
import serial
import time
import csv
import json

# 2. Capture Images (Scan)
This section operates the stereo camera of the scanner. The image, which is received as a stitched image from each camera, will be split and saved in a new timestamped directory. Images will be numbered sequentially. The camera will capture images automatically, without user input other than a start and stop command. Later prototypes of the scanner will contain a hardware button to control this feature.  

In [ ]:
current_directory = os.getcwd()
new_directory = os.path.join(current_directory, r'scans') # change to timestamp rather than static folder.
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

s = 1
if len(sys.argv) > 1:
    s = sys.argv[1]
source = cv2.VideoCapture(s)

# add a line to first wait for the "start" command from the user. Choose a key.
i = 1
while cv2.waitKey(1) != 27:  # Escape to exit. 
    has_frame, frame = source.read()
    frame = cv2.flip(frame, 0)

    if keyboard.is_pressed('s'):  # wait for 's' key to save images. #Change this to be automatic. 
        left_img = frame[:, :int(frame.shape[1] / 2), :]
        right_img = frame[:, int(frame.shape[1] / 2):, :]
        cv2.imwrite(str(new_directory) + '/left_img_' + str(i) + '.png', left_img)
        cv2.imwrite(str(new_directory) + '/right_img_' + str(i) + '.png', right_img)

        i += 1
        time.sleep(.1)  # debounce key press

    cv2.imshow('Camera', frame)
source.release()
cv2.destroyAllWindows()

# 3. Read IMU Data and Save it as JSON
Data from the MPU6090 is passed through the Arduino Nano Every who sends it to the computer over serial. When an image is captured the the scanner pose data (from the IMU) must be captured simultaneously. Therefore, the data must be saved and be corelated to its image pair. This will be done by saving JSON files with said data. This section will focus on reading the serial port, gathering the relevant data, and saving it to JSON files that are innumerated.

I need to dig in to IMUs becasue I may need to add code for correcting drift. 

In [ ]:
data_export = []

def readserial(comport, baudrate, timestamp=False):

    ser = serial.Serial(comport, baudrate, timeout=0.1)         # 1/timeout is the frequency at which the port is read

    while True:

        data = ser.readline().decode().strip()
        data_export.append(data)
        if data and timestamp:
            timestamp = time.time()
            print(f'{timestamp} > {data}')
        elif data:
            print(data)
    
    
readserial('COM4', 9600, True)                          # COM port, Baudrate, Show timestamp

# 4. Combining Code